In [1]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

In [3]:
df = pd.read_csv("./ogclass.csv")
df = df.drop(columns=["Unnamed: 0"])
df

,name,color
0,0,purple
1,1,orange
2,2,orange
3,3,white
4,4,yellow
...,...,...
9995,9995,orange
9996,9996,white
9997,9997,blue
9998,9998,orange


In [4]:
min_samples = df['color'].value_counts().min()
min_samples

933

In [5]:
from sklearn.utils import resample

balanced_df = pd.concat([resample(df[df['color'] == cls], replace=False, n_samples=min_samples) for cls in df['color'].unique()])

balanced_df


,name,color
5310,5310,purple
3444,3444,purple
2773,2773,purple
5815,5815,purple
3496,3496,purple
...,...,...
7426,7426,gray
4438,4438,gray
6796,6796,gray
5005,5005,gray


In [6]:
grouped = balanced_df.groupby('color')
sampled_df = grouped.apply(lambda x: x.sample(frac=0.1))
sampled_df = sampled_df.reset_index(drop=True)
sampled_df

,name,color
0,2760,black
1,6068,black
2,8915,black
3,987,black
4,8323,black
...,...,...
925,3482,yellow
926,5059,yellow
927,4803,yellow
928,2128,yellow


In [7]:
for fname in sampled_df.name.to_list():
    # print(f"mv ./data/{fname}.png ./stdtestset/")
    os.system(f"mv ./data/{fname}.png ./stdtestset/")

In [8]:
balanced_minus_sampled = pd.merge(balanced_df, sampled_df, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only'].drop('_merge', axis=1)
balanced_minus_sampled

,name,color
0,5310,purple
1,3444,purple
2,2773,purple
3,5815,purple
4,3496,purple
...,...,...
9325,7426,gray
9326,4438,gray
9327,6796,gray
9328,5005,gray


In [9]:
grouped_for_init = balanced_minus_sampled.groupby('color')
init_df = grouped_for_init.apply(lambda x: x.sample(frac=0.1))
init_df = init_df.reset_index(drop=True)
init_df

,name,color
0,4415,black
1,1566,black
2,6681,black
3,1896,black
4,1029,black
...,...,...
835,1012,yellow
836,6483,yellow
837,3287,yellow
838,2079,yellow


In [10]:
for fname in init_df.name.to_list():
    os.system(f"mv ./data/{fname}.png ./init_population/")

In [11]:
! mkdir ./segregated_files

mkdir: cannot create directory ‘./segregated_files’: File exists


In [16]:
df.to_csv("./segregated_files/df.csv")
# min_samples.to_csv("./segregated_files/min_samples.csv")
balanced_df.to_csv("./segregated_files/balanced_df.csv")
sampled_df.to_csv("./segregated_files/sampled_df.csv")
balanced_minus_sampled.to_csv("./segregated_files/balanced_minus_sampled.csv")
init_df.to_csv("./segregated_files/init_df.csv")